# Modificación de la ecuación de Tait, la velocidad de sonido artificial y la viscosidad.

Haciendo una revisión superficial con ChatGPT encontramos los siguientes errores pequeños que pueden producir inestabilidades numéricas y que no permiten el uso de los valores realistas de B,c. Haremos un estudio detallado de como estos valores se representan en la literatura y haremos las correcciones necesarias para llevar a cabo la implementación con valores que tengan sentido físico.

- Viscosidad artificial demasiado débil (α,β ≪ 0.1)

- c grande con Δt probablemente demasiado grande

- Uso de c constante desacoplado de la EOS

- Presión rígida en fronteras

- $\epsilon^2$ demasiado pequeño en $\phi_{ij}$


## 1. Valores de $\alpha$ y $\beta$ encontrados en la literatura.

Encontramos en Becker & Teschenner (2007) que usan un modelo lineal para la viscosidad artificial que solo depende de un parámetro $\alpha$.

$\Pi_{ab}$ =
\begin{cases}
-\nu \dfrac{\mathbf{v}_{ab}\cdot\mathbf{x}_{ab}}
{|\mathbf{x}_{ab}|^2 + \varepsilon h^2},
& \text{si } \mathbf{v}_{ab}\cdot\mathbf{x}_{ab} < 0, \\[8pt]
0,
& \text{si no}.
\end{cases}

Donde

$$ \nu = \frac{2 \alpha h c_s}{\rho_a + \rho_b} $$

Y:

$$ \alpha \in [0.08, 0.5],\ \epsilon = 0.01 $$

Hacemos los cambios necesarios para incluir esta nueva función dentro de los códigos hasta ahora trabajados.

**Según se entiende WCSPH no trabaja con la energía interna solo con las ecuaciones de conservación de la masa y del momentum**